<a href="https://colab.research.google.com/github/AniketKS30/Deep-Learning/blob/main/Hyperparameter_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 28.5 MB/s 


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name !='/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [4]:
import pandas as pd
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch

In [5]:
data = pd.read_csv("/content/drive/MyDrive/DataSet/train (1).csv")

In [6]:
data.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
57,58,0,3,"Novel, Mr. Mansouer",male,28.5,0,0,2697,7.2292,NaN,C
515,516,0,1,"Walker, Mr. William Anderson",male,47.0,0,0,36967,34.0208,D46,S
771,772,0,3,"Jensen, Mr. Niels Peder",male,48.0,0,0,350047,7.8542,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
75,76,0,3,"Moen, Mr. Sigurd Hansen",male,25.0,0,0,348123,7.6500,F G73,S


In [7]:
data.shape

(891, 12)

In [8]:
#droping the unnecessary column
data.drop(columns = ['Name','Ticket','Cabin'],inplace = True)
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [9]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [10]:
for column in ['Age','Embarked']:
    data[column].fillna(data[column].mode()[0], inplace=True)
data['Age']=data['Age'].astype('int64')

In [11]:
from sklearn.preprocessing import LabelEncoder
la = LabelEncoder()

def cat_to_num(c,data): 
  for i in range(len(c)):
    data.iloc[:,c[i]] = la.fit_transform(data.iloc[:,c[i]])
    #print('\nSkewness for Column: ',skew(data.iloc[:,c[i]]))
  return data.sample(3)

In [12]:
cat_to_num([3],data)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
366,367,1,1,0,60,1,0,75.2500,C
41,42,0,2,0,27,1,0,21.0000,S
261,262,1,3,1,3,4,2,31.3875,S


In [13]:
cat_to_num([8],data)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
224,225,1,1,1,38,1,0,90.0000,2
687,688,0,3,1,19,0,0,10.1708,2
442,443,0,3,1,25,1,0,7.7750,2


In [14]:
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [15]:
data.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.647587,28.551066,0.523008,0.381594,32.204208,1.536476
std,257.353842,0.486592,0.836071,0.477990,13.207013,1.102743,0.806057,49.693429,0.791503
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,1.000000,24.000000,0.000000,0.000000,14.454200,2.000000
75%,668.500000,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


In [16]:
x = data.drop(['Survived'], axis=1)
y = data.iloc[:,1].values
#y=data('Survived')

In [17]:
x.shape

(891, 8)

In [18]:
y.shape

(891,)

In [19]:
from sklearn.preprocessing import MinMaxScaler
mx = MinMaxScaler()

In [20]:
x = mx.fit_transform(x)
y= mx.fit_transform(y.reshape(-1,1))

In [21]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
    model.add(layers.Dense(units=hp.Int('units_'+str(i),min_value=32,
    max_value=512,step=32),
    activation='relu'))
  model.add(layers.Dense(1, activation='linear'))

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
                loss='mean_absolute_error',
                metrics=['mean_absolute_error'])
  return model

In [22]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project1',
    project_name='Power_Consumption')

In [23]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [24]:
tuner.search(x,y, epochs=5 , validation_split=0.2)

Trial 5 Complete [00h 00m 07s]
val_mean_absolute_error: 0.3581539789835612

Best val_mean_absolute_error So Far: 0.21968452632427216
Total elapsed time: 00h 00m 34s
INFO:tensorflow:Oracle triggered exit


In [25]:
tuner.results_summary()

Results summary
Results in project1/Power_Consumption
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 18
units_0: 416
units_1: 224
learning_rate: 0.0001
units_2: 512
units_3: 512
units_4: 480
units_5: 32
units_6: 352
units_7: 416
units_8: 288
units_9: 192
units_10: 160
units_11: 448
units_12: 288
units_13: 288
units_14: 160
units_15: 480
units_16: 96
units_17: 320
Score: 0.21968452632427216
Trial summary
Hyperparameters:
num_layers: 4
units_0: 96
units_1: 128
learning_rate: 0.0001
units_2: 256
units_3: 64
units_4: 256
units_5: 320
units_6: 128
units_7: 320
units_8: 288
units_9: 352
units_10: 320
units_11: 480
units_12: 480
units_13: 288
units_14: 448
units_15: 224
units_16: 448
units_17: 128
Score: 0.2860150933265686
Trial summary
Hyperparameters:
num_layers: 18
units_0: 224
units_1: 384
learning_rate: 0.01
units_2: 192
units_3: 160
units_4: 96
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
